In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , mean_absolute_error, r2_score

In [ ]:
df = pd.read_csv("/content/data.csv", encoding='ISO-8859-1')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
# Drop rows with missing CustomerID
df = df.dropna(subset=["CustomerID"])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [ ]:
# Fix negative or zero quantity/price if necessary
df = df[(df.Quantity > 0) & (df.UnitPrice > 0)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397884 non-null  object 
 1   StockCode    397884 non-null  object 
 2   Description  397884 non-null  object 
 3   Quantity     397884 non-null  int64  
 4   InvoiceDate  397884 non-null  object 
 5   UnitPrice    397884 non-null  float64
 6   CustomerID   397884 non-null  float64
 7   Country      397884 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.3+ MB


In [ ]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df['Total_Sales'] = df['Quantity'] * df['UnitPrice']

In [ ]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [ ]:
necessary_cols = [
    'InvoiceNo',
    'CustomerID',
    'InvoiceDate',
    'Quantity',
    'UnitPrice',
    'Total_Sales'
]

In [ ]:
df = df[necessary_cols]

In [ ]:
df.head()

,InvoiceNo,CustomerID,InvoiceDate,Quantity,UnitPrice,Total_Sales
0,536365,17850.0,2010-12-01 08:26:00,6,2.55,15.30
1,536365,17850.0,2010-12-01 08:26:00,6,3.39,20.34
2,536365,17850.0,2010-12-01 08:26:00,8,2.75,22.00
3,536365,17850.0,2010-12-01 08:26:00,6,3.39,20.34
4,536365,17850.0,2010-12-01 08:26:00,6,3.39,20.34


In [ ]:
import datetime as dt

df["invoice_month"] = df["InvoiceDate"].apply(
    lambda x: dt.datetime(x.year, x.month, 1)
)

In [ ]:
df.head()

,InvoiceNo,CustomerID,InvoiceDate,Quantity,UnitPrice,Total_Sales,invoice_month
0,536365,17850.0,2010-12-01 08:26:00,6,2.55,15.30,2010-12-01
1,536365,17850.0,2010-12-01 08:26:00,6,3.39,20.34,2010-12-01
2,536365,17850.0,2010-12-01 08:26:00,8,2.75,22.00,2010-12-01
3,536365,17850.0,2010-12-01 08:26:00,6,3.39,20.34,2010-12-01
4,536365,17850.0,2010-12-01 08:26:00,6,3.39,20.34,2010-12-01


In [ ]:
df.CustomerID.nunique()

4338

In [ ]:
# grouping = df.groupby("CustomerID")["invoice_month"]
# df["cohort_month"] = grouping.transform('min')

In [ ]:
snapshot_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)

In [ ]:
cutoff = pd.to_datetime("2011-11-10")

train_df = df[df.InvoiceDate <= cutoff]
target_df = df[(df.InvoiceDate > cutoff) & (df.InvoiceDate <= cutoff + pd.Timedelta(days=30))]

# Compute target per customer
target = target_df.groupby("CustomerID")["Total_Sales"].sum().reset_index()
target.rename(columns={"Total_Sales": "Future30Spend"}, inplace=True)

In [ ]:
feat = train_df.groupby("CustomerID").agg({
    "InvoiceDate": [
        lambda x: (cutoff - x.max()).days,
        "nunique"
    ],
    "InvoiceNo": "nunique",
    "Quantity": "sum",
    "UnitPrice": "mean",
    "Total_Sales": "sum"
})

# Rename columns
feat.columns = [
    "RecencyDays", "ActiveDays",
    "NumOrders", "TotalQty", "AvgUnitPrice", "TotalSpent"
]
feat = feat.reset_index()

# Merge with target
data = feat.merge(target, on="CustomerID", how="left")
data["Future30Spend"].fillna(0, inplace=True)

data.head()

/tmp/ipython-input-2577123324.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Future30Spend"].fillna(0, inplace=True)


,CustomerID,RecencyDays,ActiveDays,NumOrders,TotalQty,AvgUnitPrice,TotalSpent,Future30Spend
0,12346.0,295,1,1,74215,1.040000,77183.60,0.00
1,12347.0,9,6,6,2266,2.734912,4085.18,224.82
2,12348.0,45,4,4,2341,5.764839,1797.24,0.00
3,12350.0,280,1,1,197,3.841176,334.40,0.00
4,12352.0,6,8,8,536,15.930706,2506.04,0.00


In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=["CustomerID", "Future30Spend"])
y = data["Future30Spend"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.3f}")

MAE: 318.19
RMSE: 1407.88
R²: 0.172


In [ ]:
customer_id = 14850

# Get the customer's feature row
customer_data = data[data["CustomerID"] == customer_id] \
                    .drop(columns=["CustomerID", "Future30Spend"])

# Predict
predicted_spend = model.predict(customer_data)

print(f"Predicted Future30Spend for Customer {customer_id}: {predicted_spend[0]:.2f}")


Predicted Future30Spend for Customer 14850: 163.07
